In [3]:
import pandas as pd
from Korpora import Korpora

In [5]:
corpus = Korpora.load("nsmc")
corpus_df = pd.DataFrame(corpus.test)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\kdp\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\kdp\K

In [16]:
trainDF = corpus_df.sample(frac=0.9, random_state=42)
testDF = corpus_df.drop(trainDF.index)

In [17]:
print(trainDF.head().to_markdown)
print("Training Data Size :", len(trainDF))
print("Testing Data Size :", len(testDF))

<bound method DataFrame.to_markdown of                                                     text  label
33553  모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만...      1
9427                    무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...      0
199                                          신날 것 없는 애니.      0
12447                                              잔잔 격동      1
39489                                 오랜만에 찾은 주말의 명화의 보석      1>
Training Data Size : 45000
Testing Data Size : 5000


# 2. 단어 사전 생성
- 토큰화 진행 => 형태소 분석기 선택
- 단어 사전

## 2-1 토큰화 진행(문장 to 단어)

In [18]:
# 모듈 로딩
from konlpy.tag import Okt

# 토큰화 인스턴스 생성
tokenizer = Okt()

학습은 데이터 크기가 작은 테스트 데이터세트를 활용해서 실습 진행

In [19]:
# 문장 => 단어 분리
trainDF.text[:10]

33553    모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만...
9427                      무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...
199                                            신날 것 없는 애니.
12447                                                잔잔 격동
39489                                   오랜만에 찾은 주말의 명화의 보석
42724         영화공사중 도로를 보았는데 에어컨을.. 정말 더웠다.하지만 더글라스오빠연기 압권
10822                      재밌다. 유쾌한 노랫소리. 주인공들의 코믹한 연기. 굿굿
49498                                  음.. 괜찮네요. 생각보다 좋았어요
4144     """21세기 """"레니 할린, 작품은 """"마인트헌터, 뿐인가!? 연출이 tv...
36958    이걸 영화관에서 보다니 감동이었고 관객이 다들 혼자온 덕후들이라 더욱 감개무량했던 ...
Name: text, dtype: object

In [24]:
# 문장 => 단어 분리 
# for text in trainDF.text:
#     print(tokenizer.morphs(text, stem=True)) # stem = 어근만 
#     break

train_tokens = [tokenizer.morphs(text, stem=True) for text in trainDF.text]
test_tokens = [tokenizer.morphs(text, stem=True) for text in testDF.text]

In [33]:
print(f"[train_tokens] {len(train_tokens)}개")
print(f"[test_tokens] {len(test_tokens)}개")
print(f"[test_tokens 0번] {len(train_tokens[0])}개")
print(f"[test_tokens 0번] {len(test_tokens[0])}개")
print(f"[test_tokens 1번] {len(train_tokens[1])}개")
print(f"[test_tokens 1번] {len(test_tokens[1])}개")
# 리스트 내의 요소의 개수가 제각각인 것을 알 수 있음 

[train_tokens] 45000개
[test_tokens] 5000개
[test_tokens 0번] 19개
[test_tokens 0번] 18개
[test_tokens 1번] 14개
[test_tokens 1번] 6개


## 2-2 토큰화 => 단어/어휘 사전 생성

In [34]:
from collections import Counter

# 단어 사전 생성 함수
def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    
    # 단어/토큰에 대한 빈도수 계산
    for tokens in corpus:
        counter.update(tokens)
        
    # 단어/어휘 사전 생성
    vocab = special_tokens
    
    # 단어/어휘 사전에 빈도수가 높은 단어 추가
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

In [36]:
from konlpy.tag import Okt

vocab = build_vocab(train_tokens, 5000, ["<PAD>", "<UNK>"])
print(f"[VOCAB => {vocab[:30]}]")

[VOCAB => ['<PAD>', '<UNK>', '.', '이', '영화', '보다', '하다', '의', '..', '에', '가', '...', '을', '도', '들', ',', '는', '를', '은', '없다', '이다', '있다', '좋다', '?', '너무', '다', '정말', '한', '되다', '재밌다']]


## 2-3 인코딩 & 디코딩 인덱싱 만들기 

In [38]:
# 인코딩 : 문자 => 순자
token_to_id = {v:id for id, v in enumerate(vocab)}

# 디코딩 : 숫자 => 문자 
id_to_token = {id:v for id, v in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<PAD>', '<UNK>', '.', '이', '영화', '보다', '하다', '의', '..', '에']
5002


In [53]:
# 리뷰의 문자를 정수로 변환 및 단어 사건 어휘 사전에 없는 문자로 처리
UNK_ID = token_to_id.get("<UNK>")
train_ids = [[token_to_id.get(token, UNK_ID) for token in text] for text in train_tokens]
test_ids = [[token_to_id.get(token, UNK_ID) for token in text] for text in test_tokens] 

In [54]:
print(f"train_ids => {len(train_ids)}개")
print(f"test_ids => {len(test_ids)}개") # 5000

train_ids => 45000개
test_ids => 5000개


# 3. 데이터 가공 
- 토큰 데이터 정수 인코딩
- 데이터 길이 표준화 => 다른 길이의 데이터를 길어 맞추기(그래야 시퀀스가 통일됨) -> 1개 문장 구성하는 단어 맞추기 

## 3-1 토큰 정수화

## 3-2 데이터 구성 단어 수 맞추기 즉, 패딩
- 단어 수 선정 필요
- 선정된 단어 수에 맞게 데이터 길면 자르거, 짧으면 치워 

In [55]:
# 명선생님 ver -> 왼쪽 default
# def pad_sequences(sequences, max_length, pad_value, start="L"):
#     result = list()
#     for sequence in sequences:
#         sequence = sequence[:max_length] if start == "L" else sequence[-1*max_length:]
#         pad_length = max_length - len(sequence)
#         padded_sequence = sequence + [pad_value] * pad_length if start == "L" else  [pad_value] * pad_length + sequence
#         result.append(padded_sequence)
#     return np.asarray(result)

In [56]:
# 패딩 처리 함수
# sentence : 토큰화된 문장 데이터
# max_length : 최대 문장길이 즉, 1개 문장 구성 단어수
# pad : 패딩 처리 시 추가될 문자 값
# start : 패딩 시 처리 방향 [RL : 오른쪽 즉, 뒷부분 자르기/추가하기]
def pad_sequences(sequences, max_len, pad, start="R"):
    result = []
    for sen in sequences:
        sen = sen[:max_len] if start == "R" else sen[:-1*max_len]
        padd_sen = sen + [pad]*(max_len-len(sen)) if start == "R" else ([pad]*(max_len-len(sen))) + sen
        result.append(padd_sen)
    
    return result

In [57]:
# 학습용, 테스트용 데이터 패딩 처리
PAD_ID = token_to_id.get("<PAD>")
MAX_LENGTH = 32

train_ids = pad_sequences(train_ids, MAX_LENGTH, PAD_ID)
test_ids = pad_sequences(test_ids, MAX_LENGTH, PAD_ID)

In [58]:
print(f"train_idx => {len(train_ids[0])} 개, {train_ids[0]}")
print(f"test_idx => {len(test_ids[0])} 개")

train_idx => 32 개, [258, 1621, 12, 1370, 171, 222, 365, 4, 2, 2101, 1044, 255, 36, 15, 3993, 1, 1, 1029, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
test_idx => 32 개


# 4. 데이터 학습 준비
- 데이터 로더 준비
- 학습용/테스트용 함수
- 모델 클래스
- 학습 관련 변수 => DEVICE, OPTIMIZER, MODEL 인스턴스, EPOCHS, BATCH_SIZE, LOSS_FN

## 4-1 데이터 로더 준비

In [62]:
from torch.utils.data import TensorDataset, DataLoader
import torch

In [74]:
# 데이터셋 생성 : List => Tensor 
# train_ids => 토큰이 정수로 바뀐 상태 List
# Label => trainDF.label.values

dataTS = torch.LongTensor(train_ids)
labelTS = torch.FloatTensor(trainDF.label.values)

print(f"dataTS => {dataTS.shape}, labelTS => {labelTS.shape}")

trainDS = TensorDataset(dataTS, labelTS) # train 데이터셋 생성

dataTS => torch.Size([45000, 32]), labelTS => torch.Size([45000])


In [75]:
# 학습용 데이터셋
dataTS_ = torch.LongTensor(test_ids)
labelTS_ = torch.FloatTensor(testDF.label.values)

print(f"dataTS => {dataTS_.shape}, labelTS => {labelTS_.shape}")

testDS = TensorDataset(dataTS_, labelTS_)

dataTS => torch.Size([5000, 32]), labelTS => torch.Size([5000])


In [76]:
# 데이터로더 생성
BATCH_SIZE = 32
trainDL = DataLoader(trainDS, BATCH_SIZE, shuffle=True)
testDL = DataLoader(testDS, BATCH_SIZE, shuffle=True)

## 4-2 모델 클래스 정의
- 입력층 : Embedding Layer
- 은닉층 : RNN/LSTM Layer
- 은닉층 : dropout Layer
- 출력층 : Linear Layer

In [77]:
from torch import nn

class SentenceClassfier(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, n_layers, dropout=0.5, bidirectional=True, model_type="lstm"):
        super().__init__()
        
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab, # 피쳐 개수
            embedding_dim=embedding_dim, # 줄이고 나서 피쳐 개수(진짜 내 맘대로)
            padding_idx=0
        )
        
        if model_type == "rnn":
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
            
        elif model_type == "lstm":
            self.model = nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        
        # 양방향이면 방향성이 2이라서 2곱해주고
        # 단방향이면 1이라서 1곱해줌(1잉께 생략 가능)
        if bidirectional: 
            self.classifier = nn.Linear(hidden_dim * 2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim * 1, 1)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [78]:
from torch import optim

n_vocab = len(token_to_id) # 피쳐개수 = 단어 개수
hidden_dim = 64
embedding_dim = 128
n_layers = 2

device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = SentenceClassfier(n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

In [79]:
import numpy as np

def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f"Train Loss {step} : {np.mean(losses)}")


def test(model, datasets, criterion, device):
    model.eval()
    losses = list()
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits)>.5
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )

    print(f"Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}")


epochs = 5
interval = 500

for epoch in range(epochs):
    train(classifier, trainDL, criterion, optimizer, device, interval)
    test(classifier, testDL, criterion, device)

Train Loss 0 : 0.7076765298843384
Train Loss 500 : 0.6832133019993643
Train Loss 1000 : 0.6494185156100518
Val Loss : 0.5431304589198653, Val Accuracy : 0.7412
Train Loss 0 : 0.5376872420310974
Train Loss 500 : 0.4924626992848105
Train Loss 1000 : 0.4769163194980536
Val Loss : 0.4179986618506681, Val Accuracy : 0.8118
Train Loss 0 : 0.3111460208892822
Train Loss 500 : 0.38709146759943097
Train Loss 1000 : 0.3788888713011851
Val Loss : 0.39128512922365954, Val Accuracy : 0.822
Train Loss 0 : 0.5061667561531067
Train Loss 500 : 0.33768229828980156
Train Loss 1000 : 0.34008305528751026
Val Loss : 0.3947492801839379, Val Accuracy : 0.8248
Train Loss 0 : 0.24726185202598572
Train Loss 500 : 0.3133950291458004
Train Loss 1000 : 0.31034168120209393
Val Loss : 0.3969756348687372, Val Accuracy : 0.8278
